In [ ]:
from collections import defaultdict
import numpy as np
from pychunkedgraph.graph import ChunkedGraph
from pychunkedgraph.graph.utils.inspect import dfs_print_node
from pychunkedgraph.graph import cache

from pychunkedgraph.graph.edits import add_edges, remove_edges
from pychunkedgraph.graph.utils.context_managers import TimeIt
from pychunkedgraph.graph.operation import MulticutOperation
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [ ]:
data = {
    "sources":[["94525015244048934",909265.0625,941857.625,784103.25]],
    "sinks":[["94595383988237777",909356.9375,941785,784103.25]]
}
data_dict = {}
for k in ["sources", "sinks"]:
    data_dict[k] = defaultdict(list)
    for node in data[k]:
        node_id = node[0]
        x, y, z = node[1:]
        coordinate = np.array([x, y, z]) / cg.meta.resolution
        atomic_id = cg.get_atomic_id_from_coord(
            coordinate[0],
            coordinate[1],
            coordinate[2],
            parent_id=np.uint64(node_id),
        )

        if atomic_id is None:
            raise ValueError("aha")
        data_dict[k]["id"].append(atomic_id)
        data_dict[k]["coord"].append(coordinate)

edges = [
    [data_dict["sources"]["id"][0], data_dict["sinks"]["id"][0]],
]
roots_old = cg.get_roots(edges[0])
parents_old = cg.get_parents(edges[0])
print(cg.get_cross_chunk_edges_layer(edges))
print(edges[0])
print(parents_old)
print(roots_old)
# dfs_print_node(cg, roots_old[0, limit=2, sep="    ", count_only=True)

In [ ]:
with TimeIt("remove edges"):
    result = cg.remove_edges(
        user_id="hi",
        atomic_edges=None,
        source_coords=data_dict["sources"]["coord"],
        sink_coords=data_dict["sinks"]["coord"],
        source_ids=data_dict["sources"]["id"],
        sink_ids=data_dict["sinks"]["id"],        
    )

In [ ]:
print(result.new_root_ids)
roots = result.new_root_ids

roots = [864691139376503998, 864691139574208201]

dfs_print_node(cg, 864691139312626617, limit=2, sep="    ", count_only=True)
dfs_print_node(cg, 864691138079759249, limit=2, sep="    ", count_only=True)

dfs_print_node(cg, roots[0], limit=2, sep="    ", count_only=True)
dfs_print_node(cg, roots[1], limit=2, sep="    ", count_only=True)